In [1]:
import numpy as np
import pandas as pd
import re
import datetime
racial = pd.read_csv("../notebooks/racialdata.csv", encoding= 'unicode_escape')

In [2]:
june_2019 = pd.read_csv("../notebooks/Crowd Estimates June 2019-edit.csv", encoding= 'unicode_escape')
july_2019 = pd.read_csv("../notebooks/Crowd Estimates July 2019-edit.csv", encoding= 'unicode_escape')
august_2019 = pd.read_csv("../notebooks/Crowd Estimates August 2019-edit.csv", encoding= 'unicode_escape')
september_2019 = pd.read_csv("../notebooks/Crowd Estimates September 2019-edit.csv", encoding= 'unicode_escape')
october_2019 = pd.read_csv("../notebooks/Crowd Estimates October 2019-edit.csv", encoding= 'unicode_escape')
november_2019 = pd.read_csv("../notebooks/Crowd Estimates November 2019-edit.csv", encoding= 'unicode_escape')
december_2019 = pd.read_csv("../notebooks/Crowd Estimates December 2019-edit.csv", encoding= 'unicode_escape')
january_2020 = pd.read_csv("../notebooks/Crowd Estimates January 2020-edit.csv", encoding= 'unicode_escape')
march_2020 = pd.read_csv("../notebooks/Crowd Estimates March 2020-edit.csv", encoding= 'unicode_escape')
april_2020 = pd.read_csv("../notebooks/Crowd Estimates April 2020-edit.csv", encoding= 'unicode_escape')

september_2019['Pro(2)/Anti(1)'] = september_2019['Pro(2)/Anti(1)'].map({'2':'2','1':'1','0':'0','2.':'2','1.':'1','0.':'0'})
september_2019['ReportedArrests'] = september_2019['ReportedArrests'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan})
september_2019['ReportedParticipantInjuries'] = september_2019['ReportedParticipantInjuries'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan})
september_2019['ReportedPoliceInjuries'] = september_2019['ReportedPoliceInjuries'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan})
september_2019['ReportedPropertyDamage'] = september_2019['ReportedPropertyDamage'].replace({'nan':np.nan, 'DEADLINK':np.nan,
                                                                              'DEAD LINK':np.nan,'DEADLINK, doesn\'t allow access to article':np.nan,
                                                                                            'DEADLINK\t':np.nan})
#imported only the ones under tally and not the individual ones specifically for certain marches - like women's rights
#and light for life

In [3]:
def add_city(x):
    first = x.columns[0]
    x.rename(columns = {first: "City"}, inplace = True)
    drop_city = x[x['City'].isnull() == True].index
    x.drop(drop_city, inplace = True)
    x = x.loc[:,['City','Location','County','StateTerritory','Country','Date','EstimateText','EstimateLow','BestGuess','EstimateHigh','AdjustedLow',
  'AdjustedHigh','Actor','Claim','Pro(2)/Anti(1)','EventType','ReportedArrests','ReportedParticipantInjuries',
  'ReportedPoliceInjuries','ReportedPropertyDamage','TownsCities','Events']]
    x['Location'] = x['City']
    for y in list(x.index):
        city = x.loc[y, 'City']
        state = x.loc[y, 'StateTerritory']
        x.loc[y, 'Location'] = str(city) + ', ' + str(state)
    x['Event'] = x['Claim']
    for y in list(x.index):
        if ';' in str(x.loc[y,'Claim']):
            split = str(x.loc[y,'Claim']).split(';')
            if len(split) == 2:
                event = split[0]
                claim = split[1]
                x.loc[y,'Claim'] = claim
                x.loc[y,'Event'] = event
            else:
                both = split[0]
                x.loc[y,'Claim'] = both
                x.loc[y,'Event'] = both
    #trying to split to see if we can divide them on claim and event, because there is an event column in the big
    #data set, but some of the claims are very weird - it worked for june_2019, tried to just make the 
    #split a string item but that didn't work either
    x = x.drop(columns = ['City', 'StateTerritory', 'Events', 'TownsCities', 'Country', 
                      'EstimateText', 'BestGuess'])
    x['Date'] = pd.to_datetime(x["Date"])
    x['Pro(2)/Anti(1)'] = pd.to_numeric(x['Pro(2)/Anti(1)'])
    x['ReportedArrests'] = pd.to_numeric(x['ReportedArrests']) 
    x['ReportedParticipantInjuries'] = pd.to_numeric(x['ReportedParticipantInjuries'])
    x['ReportedPoliceInjuries'] = pd.to_numeric(x['ReportedPoliceInjuries'])
    x['ReportedPropertyDamage'] = pd.to_numeric(x['ReportedPropertyDamage'])
    return x

In [4]:
july_2019new = add_city(july_2019)
august_2019new  = add_city(august_2019)
september_2019new  = add_city(september_2019)
october_2019new = add_city(october_2019)
november_2019new = add_city(november_2019) 
december_2019new = add_city(december_2019) 
january_2020new = add_city(january_2020) 
march_2020new = add_city(march_2020) 
april_2020new = add_city(april_2020)
june_2019new = add_city(june_2019)

In [5]:
racial['Date'] = pd.to_datetime(racial["Date"])
months_new = [june_2019new, july_2019new, august_2019new, september_2019new, october_2019new, november_2019new, 
          december_2019new, january_2020new, march_2020new ,april_2020new]
#no september
racial = racial.merge(june_2019new, on = ['Location', 'Date'], how = 'left')



#september_2019new

                                    

In [6]:
['Location', 'County','Date', 'EstimateLow','EstimateHigh', 'AdjustedLow','AdjustedHigh','Actor',
 'Claim','Pro(2)/Anti(1)','EventType','ReportedArrests','ReportedParticipantInjuries','ReportedPoliceInjuries',
 'ReportedPropertyDamage','Event']
#print(september_2019new.dtypes)
racial.dtypes
#september dtypes aren't changing properly

Date                           datetime64[ns]
Location                               object
Attendees                             float64
Event (legacy; see tags)               object
Tags                                   object
Curated                                object
Source                                 object
Total Articles                          int64
County                                 object
EstimateLow                           float64
EstimateHigh                          float64
AdjustedLow                           float64
AdjustedHigh                          float64
Actor                                  object
Claim                                  object
Pro(2)/Anti(1)                        float64
EventType                              object
ReportedArrests                       float64
ReportedParticipantInjuries           float64
ReportedPoliceInjuries                float64
ReportedPropertyDamage                float64
Event                             

In [7]:
months_new = [june_2019new, july_2019new, august_2019new, october_2019new, november_2019new, 
          december_2019new, january_2020new, march_2020new ,april_2020new]
#september_2019new - removed

for x in months_new:
    racial = racial.merge(x, on = ['Location', 'Date','County', 'EstimateLow','EstimateHigh', 'AdjustedLow','AdjustedHigh','Actor',
 'Claim','Pro(2)/Anti(1)','EventType','ReportedArrests','ReportedParticipantInjuries','ReportedPoliceInjuries',
 'ReportedPropertyDamage','Event'], how = 'left')
    #all other ones merge properly


In [8]:
#next step to take
n = racial[(racial['Attendees'].isnull() == True) & (racial['EstimateHigh'].isnull() == True)].index
racial.drop(n, inplace = True)

index_attendee = racial[(racial['Attendees'].isnull() == True) & (racial['EstimateHigh'].isnull() == False)].index
for x in range(len(index_attendee)):
    racial.loc[index_attendee[x], 'Attendees'] = racial.loc[index_attendee[x], 'EstimateHigh']

In [9]:
racial.drop(columns=['Source','Curated', 'Total Articles','EstimateLow','EstimateHigh', 'AdjustedLow', 'AdjustedHigh'],inplace = True)

In [10]:
racial.head()

,Date,Location,Attendees,Event (legacy; see tags),Tags,County,Actor,Claim,Pro(2)/Anti(1),EventType,ReportedArrests,ReportedParticipantInjuries,ReportedPoliceInjuries,ReportedPropertyDamage,Event
0,2017-01-15,"Bowie State University, Bowie, MD",1500.0,Healthcare,Healthcare; For Affordable Care Act,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-16,"Johnson City, TN",300.0,Civil Rights,Civil Rights; For racial justice; Martin Luthe...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-01-16,"Indianapolis, IN",20.0,Environment,Environment; For wilderness preservation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-01-18,"Hartford, CT",300.0,Healthcare (Pro-Planned Parenthood),Healthcare; For Planned Parenthood,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2017-01-20,"Westlake Park, Seattle, WA",100.0,Executive,Executive; Against president,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Now need to select just the locations that we're interested in

In [11]:
def state(place):
    new = str(place).split(',')
    if len(new) == 3:
        state = re.sub(r"^\s+", "", new[2])
        return state
    elif len(new) == 2:
        state = re.sub(r"^\s+", "", new[1])
        return state
    else:
        return new[0]
racial['State'] = racial.apply(lambda row: state(row['Location']), axis=1)

In [12]:
def city(place):
    new = str(place).split(',')
    if len(new) == 3:
        city = re.sub(r"^\s+", "", new[1])
        return city
    elif len(new) == 2:
        return new[0]
    else:
        return np.nan
racial['City'] = racial.apply(lambda row: city(row['Location']), axis=1)

In [13]:
def specific(place):
    new = str(place).split(',')
    if len(new) == 3:
        return new[0]
    else:
        return np.nan
racial['Specific Location'] = racial.apply(lambda row: specific(row['Location']), axis=1)

In [14]:
locations = ['New York','Washington','Portland', 'Los Angeles','San Francisco', 'Boston', 'Chicago','Philadelphia', 'Springfield',
'Austin','Salt Lake City','Seattle','Detroit','Phoenix', 'San Diego','Sacramento']    
def include(city):
    if city in locations:
        return 'True'
    else:
        return 'False'
racial['Include'] = racial.apply(lambda row: include(row['City']), axis = 1)      

In [15]:
data = racial[racial['Include'] == 'True']

In [16]:
d2017 = datetime.datetime(2017, 12, 31)
data_2017 = data[data['Date'] <= d2017] #782 rows 
d2018 = datetime.datetime(2018, 12, 31)
data_2018 = data[(data['Date'] <= d2018) & (data['Date'] > d2017)] #769 rows 
d2019 = datetime.datetime(2019, 12, 31)
data_2019 = data[(data['Date'] <= d2019) & (data['Date'] > d2018)] #747 rows 
d2020 = datetime.datetime(2020, 12, 31)
data_2020 = data[(data['Date'] <= d2020) & (data['Date'] > d2019)] #903 rows

In [17]:
d = data[data['County'].isnull() != True]
d.head(10)

,Date,Location,Attendees,Event (legacy; see tags),Tags,County,Actor,Claim,Pro(2)/Anti(1),EventType,ReportedArrests,ReportedParticipantInjuries,ReportedPoliceInjuries,ReportedPropertyDamage,Event,State,City,Specific Location,Include
18406,2019-06-01,"Washington, DC",24.0,Other,Other; Against violence,DC,Southeast residents/general protesters,Other,1.0,rally,0.0,0.0,0.0,0.0,Other,DC,Washington,NaN,True
18424,2019-06-02,"New York, NY",100.0,Other,Other; For nudity; Facebook,New York,Grab Them By the Ballot,For nudity,0.0,sit in/rally,0.0,0.0,0.0,0.0,Other,NY,New York,NaN,True
18459,2019-06-05,"Salt Lake City, UT",24.0,Environment,Environment; Against hazardous conditions; For...,Salt Lake,Civil Riot,Environment,0.0,protest/takeover,1.0,0.0,0.0,0.0,Environment,UT,Salt Lake City,NaN,True
18465,2019-06-06,"Washington, DC",20.0,Other,Other; Against violence,DC,Young protestors,Marching for Maurice Scott after he was shot,1.0,March,0.0,0.0,0.0,0.0,Marching for Maurice Scott after he was shot,DC,Washington,NaN,True
18485,2019-06-07,"Philadelphia, PA",150.0,Racial Injustice,Civil Rights; For racial justice; For greater ...,Philadelphia,general protestors,Against alleged racist Facebook posts from police,0.0,Protest,0.0,0.0,0.0,0.0,Against alleged racist Facebook posts from police,PA,Philadelphia,NaN,True
18489,2019-06-07,"Phoenix, AZ",20.0,Racial Injustice,Civil Rights; For racial justice; For greater ...,Maricopa,Arizona chapter of Black Lives Matter,Against police shootings,1.0,Protest,0.0,0.0,0.0,0.0,Against police shootings,AZ,Phoenix,NaN,True
18491,2019-06-07,"Springfield, OR",20.0,Healthcare,Healthcare; Against closure/relocation,Lane,general protestors,Support PeaceHealth Nurse Midwifery Birth Center,0.0,Rally,0.0,0.0,0.0,0.0,Support PeaceHealth Nurse Midwifery Birth Center,OR,Springfield,NaN,True
18497,2019-06-08,"Springfield, MO",100.0,Civil Rights,Civil Rights; For abortion rights,Greene,general protestors,Against abortion restrictions,1.0,Rally,0.0,0.0,0.0,0.0,Against abortion restrictions,MO,Springfield,NaN,True
18499,2019-06-08,"Boston, MA",10000.0,Civil Rights,Civil Rights; Pro-LGBTQ; Pride,Suffolk,general supporters,LGBTQ+ pride,1.0,Parade; rally,0.0,0.0,0.0,0.0,LGBTQ+ pride,MA,Boston,NaN,True
18526,2019-06-08,"Springfield, MO",100.0,Civil Rights,Civil Rights; Against abortion rights; Counter...,Greene,general protestors,Against abortion restrictions,1.0,Rally,0.0,0.0,0.0,0.0,Against abortion restrictions,MO,Springfield,NaN,True


In [18]:
data_2020['Event (legacy; see tags)'].value_counts()

Racial Injustice           536
Other                       94
Healthcare                  74
Civil Rights                43
Executive                   41
International Relations     39
Collective Bargaining       28
Education                   23
Immigration                 12
Environment                  7
Guns                         6
Name: Event (legacy; see tags), dtype: int64

In [19]:
data_2020['Tags'].value_counts()[:20]

Civil Rights; For racial justice; For greater accountability; Police                             449
Healthcare; Against pandemic intervention; Coronavirus                                            62
International; Against war; Iran                                                                  23
Civil Rights; For women's rights; Women's March                                                   12
Education; Against hazardous conditions; Coronavirus                                              12
Civil Rights; For racial justice; For greater accountability; For freedom of assembly; Police     11
Other; For supporting police                                                                      11
Other; Against eviction; Coronavirus                                                               9
Immigration; For compassionate immigration                                                         8
Civil Rights; For racial justice; Martin Luther King, Jr.                                  

In [20]:
data_2019['Event'].value_counts()

Other                                                             3
Pride                                                             3
LGBTQ+ pride                                                      3
For the inclusion of police in an LGBTQ+ Pride event              2
pro-impeachment                                                   2
Protest police participation in Sacramento Pride                  2
oppose calls to dismantle Office of Personnel Management          2
Against abortion restrictions                                     2
against corporate involvement in SF pride parade                  2
demand climate debate among Democratic presidential contenders    2
Civil Rights                                                      1
Environment                                                       1
Marching for Maurice Scott after he was shot                      1
seek more bike lanes                                              1
protest energy company policy                   